In [1]:
import torch
import copy
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
from torch.nn import functional as F
 
import argparse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:

green_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\green'
yellow_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\yellow'
red_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\red'
blue_dir = f'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\blue'

green_set = datasets.ImageFolder(
    green_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
yellow_set = datasets.ImageFolder(
    yellow_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
red_set = datasets.ImageFolder(
    red_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
blue_set = datasets.ImageFolder(
    blue_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

train_set, test_set = torch.utils.data.random_split(green_set, [500, 100])
train_set1, train_set2 = torch.utils.data.random_split(train_set, [350, 150])

normal_set = train_set1 + train_set2
abnormal_set = red_set + yellow_set + blue_set

In [4]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        # Encoder
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        
        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))

        # Decoder        
        self.tran_cnn_layer1 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size = 2, stride = 2, padding=0),
            nn.ReLU())

        self.tran_cnn_layer2 = nn.Sequential(
            nn.ConvTranspose2d(32, 3, kernel_size = 2, stride = 2, padding=0),
            nn.Sigmoid())
            
            
    def forward(self, x):
        output = self.cnn_layer1(x)
        output = self.cnn_layer2(output)      
        output = self.tran_cnn_layer1(output)
        output = self.tran_cnn_layer2(output)
        
        return output

In [5]:
def score(model, ):    
    normal_err = 0
    abnormal_err = 0    
    
    for i, (X_normal, _) in enumerate(normal_loader):
        X_normal = X_normal.cuda()
        output = model(X_normal)
        loss = criterion(X_normal, output)
        
        if loss.item() > x:
            normal_err += 1
        
    for i, (X_abnormal, _) in enumerate(abnormal_loader):
        X_abnormal = X_abnormal.cuda()
        output = model(X_abnormal)
        loss = criterion(X_abnormal, output)
        
        if loss.item() < x:
            abnormal_err += 1
      
    err = ((normal_err/len(normal_loader)) + (abnormal_err/len(abnormal_loader)))/2.0
    
    return err